In [2]:
# ======================================================
# 🪄 STEP 1 — SETUP DJANGO ENVIRONMENT
# ======================================================

import os
import django

os.chdir('/home/ifiokambrose/Desktop/myprojects/BBIFK/')

# Set Django settings module
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'jewelryshop.settings')

# Initialize Django
django.setup()

print("✅ Django environment loaded.")


✅ Django environment loaded.


In [3]:
# ======================================================
# 🧱 STEP 2 — IMPORT MODELS AND DEPENDENCIES
# ======================================================

from store.models import Product, Category
from django.utils.text import slugify
from django.core.files import File
from decimal import Decimal
import os

print("✅ Models imported successfully.")


✅ Models imported successfully.


In [9]:
# ======================================================
# 🚀 STEP 4 — UPLOAD ALL IMAGES IN ONE FOLDER
# ======================================================

import os
from decimal import Decimal
from django.core.files import File
from django.utils.text import slugify
from store.models import Product, Category

# Allow Django ORM inside async Jupyter environment
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

BASE_IMAGE_DIR = '/home/ifiokambrose/Pictures/items/shoes'
CATEGORY_NAME = 'Oxfords'

def upload_all_products_from_dir():
    total_uploaded = 0

    # Create or get category
    category, _ = Category.objects.get_or_create(title=CATEGORY_NAME)

    # List all images in the directory
    for filename in os.listdir(BASE_IMAGE_DIR):
        if not filename.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.webp')):
            continue

        image_path = os.path.join(BASE_IMAGE_DIR, filename)
        title = os.path.splitext(filename)[0].replace('_', ' ').title()
        slug = slugify(title)
        sku = f"SKU-{slug[:8]}"

        with open(image_path, 'rb') as f:
            product = Product.objects.create(
                title=title,
                slug=slug,
                sku=sku,
                short_description=f"Auto-imported product: {title}",
                detail_description="Uploaded automatically from local directory.",
                price=Decimal("80900"),  # set desired price
                category=category,
                is_active=True,
                is_featured=False,
            )
            product.temp_image.save(filename, File(f), save=True)

        total_uploaded += 1
        print(f"✅ Uploaded: {title}")

    print(f"\n🎉 Upload complete! Total products added: {total_uploaded}")



In [10]:
upload_all_products_from_dir()


✅ Uploaded: Black Oxfords
✅ Uploaded: Brown Oxfords
✅ Uploaded: Bruno Marc
✅ Uploaded: Deep Blue Oxfords

🎉 Upload complete! Total products added: 4
